In [1]:
import csv
import os
import json
import time
import random


In [2]:
with open("/home/zjusst/qms/webarena/result_stage_3_generation/flitered_triplets.csv", "r") as f:
    reader = csv.reader(f)
    # 跳过第一行
    next(reader)
    triplets = [row for row in reader]
print(len(triplets))
print(triplets[0])

1998
["('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_36.json', 0, 0, 'http://reddit.com/')", "click [20] where [20] is button 'Sort by: Hot' hasPopup: menu expanded: False", 'The user wants to sort posts by a different criterion to better organize or view the content on the homepage.', "('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_36.json', 0, 1, 'http://reddit.com/')"]


In [3]:
# import networkx as nx
# import numpy as np

# def count_all_paths(graph_data, path_length):
#     """
#     使用邻接矩阵幂统计图中所有长度为path_length的路径数量。

#     Args:
#         graph_data (list of tuples): 你的图数据，格式为 (source_id, action, intent, target_id)。
#         path_length (int): 你想要统计的路径长度 (n)。

#     Returns:
#         dict: 一个字典，键是路径长度，值是该长度的路径总数。
#     """
#     # 1. 使用NetworkX构建一个有向图，两个节点间可以有多个边
#     G = nx.MultiDiGraph()
#     # 先添加所有节点，以确保节点顺序是固定的
#     nodes = sorted(list(set([d[0] for d in graph_data] + [d[3] for d in graph_data])))
#     G.add_nodes_from(nodes)
    
#     # 添加边
#     edges = [(d[0], d[3]) for d in graph_data]
#     G.add_edges_from(edges)
    
#     # 2. 获取邻接矩阵 A
#     # nodelist参数确保矩阵的行/列顺序是固定的
#     A = nx.to_numpy_array(G, nodelist=nodes)
    
    
#     # A_k 将存储 A 的 k 次幂
#     A_k = np.copy(A) 

#     # 3. 循环计算从1到n的路径
#     for k in range(path_length - 1):
#         A_k = A_k @ A

#     # 矩阵中所有元素的和就是长度为k的路径总数
#     count_k = int(np.sum(A_k))
#     return count_k

# # ==================== 示例用法 ====================
# if __name__ == "__main__":
#     # 你的图数据 (包含一个环路 A -> B -> C -> A 来展示路径爆炸)
#     # graph_tuples = [
#     #     ('PageA', 'action', 'intent', 'PageB'),
#     #     ('PageB', 'action', 'intent', 'PageC'),
#     #     ('PageC', 'action', 'intent', 'PageD'),
#     #     ('PageC', 'action', 'intent', 'PageA'), # 环路
#     #     ('PageD', 'action', 'intent', 'PageE'),
#     #     #('PageE', 'action', 'intent', 'PageC'),
#     # ]
#     graph_tuples=triplets
#     path_cnts = []
#     # 我们来统计长度从1到5的路径
#     max_n = 20
#     for i in range(1, max_n + 1):
#         count = count_all_paths(graph_tuples, i)
#         print(f"长度为 {i} 的路径总数:\t{count}")
#         path_cnts.append(count)

In [4]:
# # 画一个折线图
# import matplotlib.pyplot as plt


# # 改为指数坐标，每个点上显示路径数量
# plt.figure(figsize=(10, 6))
# plt.semilogy(range(1, max_n + 1), path_cnts, marker='o', linestyle='-', color='b')
# plt.title('path length vs path count')
# plt.xlabel('path length')
# plt.ylabel('path count')

In [3]:
import networkx as nx
import random

def sample_random_path(graph_data, path_length, start_node=None):
    """
    在图上进行随机游走，以采样一条指定长度的路径。
    此版本使用 MultiDiGraph 来正确处理节点间的平行边。

    Args:
        graph_data (list of tuples): 你的图数据，格式为 (source_id, action, intent, target_id)。
        path_length (int): 期望的路径长度 n (即边的数量)。
        start_node (any, optional): 路径的起始节点。如果为None，则从图中随机选择一个起点。

    Returns:
        list of tuples or None: 包含路径中每一步的元组列表，代表采样的轨迹。
                                如果起点无效或图为空，则返回None。
    """
    if not graph_data:
        print("错误：图数据为空。")
        return None

    # 1. 构建一个有向多重图 (MultiDiGraph)，存储完整的边信息
    G = nx.MultiDiGraph()
    for source, action, intent, target in graph_data:
        G.add_edge(source, target, action=action, intent=intent)

    # 2. 确定起始节点
    if start_node is not None:
        if not G.has_node(start_node):
            raise ValueError(f"错误：指定的起始节点 '{start_node}' 不在图中。")
        current_node = start_node
    else:
        # 从所有节点中随机选择一个
        current_node = random.choice(list(G.nodes()))
    
    # 3. 执行随机游走
    trajectory = []
    print(f"随机游走开始，起点: '{current_node}'，期望长度: {path_length}")

    for i in range(path_length):
        # 获取当前节点的所有出射边 (source, target, key)
        # 这是与DiGraph版本最核心的区别
        outgoing_edges = list(G.out_edges(current_node, keys=True))
        
        # 检查是否遇到“死胡同”
        if not outgoing_edges:
            print(f"警告：在第 {i+1} 步遇到死胡同（节点 '{current_node}' 没有出边）。路径提前终止。")
            break
        
        # 随机选择一条出射边
        source_node, next_node, edge_key = random.choice(outgoing_edges)
        
        # 获取被选中边的属性信息
        # 现在我们使用 u, v, key 来精确获取
        edge_data = G.get_edge_data(source_node, next_node, key=edge_key)
        
        # 记录这一步
        step_tuple = (current_node, edge_data['action'], edge_data['intent'], next_node)
        trajectory.append(step_tuple)
        
        # 更新当前节点
        current_node = next_node
        
    return trajectory

# ==================== 示例用法 ====================
if __name__ == "__main__":
    graph_tuples = triplets
    
    path_n = 3

    random_trajectory = sample_random_path(graph_tuples, path_length=path_n, start_node=None)
    
    if random_trajectory:
        print(f"\n采样到的轨迹 (实际长度 {len(random_trajectory)}):")
        for step in random_trajectory:
            print(f"  {step[0]} -> {step[3]} (动作: {step[1]}, 意图: {step[2]})")

随机游走开始，起点: '('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_1.json', 4, 1, 'http://reddit.com/create_forum')'，期望长度: 3

采样到的轨迹 (实际长度 3):
  ('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_1.json', 4, 1, 'http://reddit.com/create_forum') -> ('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_1.json', 5, 1, 'http://reddit.com/create_forum') (动作: click [1299] where [1299] is checkbox 'Formatting help +' checked: false, 意图: The user was trying to access formatting help information for creating a new forum, but the action did not result in any visible change.)
  ('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_1.json', 5, 1, 'http://reddit.com/create_forum') -> ('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_1.json', 6, 1, 'http://reddit.com/create_forum') (动作: type [1297] [Example Sidebar Content ] where [1297] is textb

In [8]:
start_point_set = set()

for triplet in triplets:
    file_name, index, is_after, real_rul = eval(triplet[0])
    if index == 0 and is_after == 0:
        start_point_set.add(triplet[0])

# 转 list
start_point_list = list(start_point_set)
start_point_list


["('812_42.json', 0, 0, 'http://reddit.com/')",
 "('812_39.json', 0, 0, 'http://reddit.com/')",
 "('812_23.json', 0, 0, 'http://reddit.com/')",
 "('812_29.json', 0, 0, 'http://reddit.com/')",
 "('812_21.json', 0, 0, 'http://reddit.com/')",
 "('812_41.json', 0, 0, 'http://reddit.com/')",
 "('812_38.json', 0, 0, 'http://reddit.com/')",
 "('812_35.json', 0, 0, 'http://reddit.com/')",
 "('812_40.json', 0, 0, 'http://reddit.com/')",
 "('812_51.json', 0, 0, 'http://reddit.com/')",
 "('812_11.json', 0, 0, 'http://reddit.com/')",
 "('812_8.json', 0, 0, 'http://reddit.com/')",
 "('812_43.json', 0, 0, 'http://reddit.com/')",
 "('812_6.json', 0, 0, 'http://reddit.com/')",
 "('812_55.json', 0, 0, 'http://reddit.com/')",
 "('812_47.json', 0, 0, 'http://reddit.com/')",
 "('812_36.json', 0, 0, 'http://reddit.com/')",
 "('812_30.json', 0, 0, 'http://reddit.com/')",
 "('812_25.json', 0, 0, 'http://reddit.com/')",
 "('812_18.json', 0, 0, 'http://reddit.com/')",
 "('812_37.json', 0, 0, 'http://reddit.com

In [12]:
import random

out_list = []
collect_num = 25

for traj_len in [2,3,4,5]:
    for i in range(collect_num):


        while True:
            if i < collect_num*0.3:
                random_trajectory = sample_random_path(graph_tuples, path_length=traj_len, start_node=random.choice(start_point_list))
            else:
                random_trajectory = sample_random_path(graph_tuples, path_length=traj_len, start_node=None)

            if len(random_trajectory) == traj_len:
                break

        temp_dict = {
            "selected_trajectory": random_trajectory,
        }
        out_list.append(temp_dict)

out_list[0]
len(out_list)

随机游走开始，起点: '('812_6.json', 0, 0, 'http://reddit.com/')'，期望长度: 2
随机游走开始，起点: '('812_1.json', 0, 0, 'http://reddit.com/')'，期望长度: 2
随机游走开始，起点: '('812_1.json', 0, 0, 'http://reddit.com/')'，期望长度: 2
随机游走开始，起点: '('812_25.json', 0, 0, 'http://reddit.com/')'，期望长度: 2
随机游走开始，起点: '('812_8.json', 0, 0, 'http://reddit.com/')'，期望长度: 2
随机游走开始，起点: '('812_40.json', 0, 0, 'http://reddit.com/')'，期望长度: 2
随机游走开始，起点: '('812_38.json', 0, 0, 'http://reddit.com/')'，期望长度: 2
随机游走开始，起点: '('812_47.json', 0, 0, 'http://reddit.com/')'，期望长度: 2
随机游走开始，起点: '('812_5.json', 11, 1, 'http://reddit.com/user/MarvelsGrantMan136/preferences')'，期望长度: 2
随机游走开始，起点: '('812_9.json', 18, 1, 'http://reddit.com/user/andrewmyles/submissions')'，期望长度: 2
随机游走开始，起点: '('812_43.json', 14, 1, 'http://reddit.com/f/AskReddit')'，期望长度: 2
随机游走开始，起点: '('812_47.json', 12, 1, 'http://reddit.com/wiki/_history/index')'，期望长度: 2
随机游走开始，起点: '('812_43.json', 16, 1, 'http://reddit.com/f/AskReddit')'，期望长度: 2
随机游走开始，起点: '('812_22.json', 13, 1, 'http://reddit.co

100

In [13]:
# 将 out_list 导出到 json 文件, /home/zjusst/qms/webarena/result_stage_3_generation/random_selected_trajs.json
with open("/home/zjusst/qms/webarena/result_stage_3_generation/random_selected_trajs.json", "w") as f:
    json.dump(out_list, f)
